*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing import sequence

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
"""
Setting some constants
"""

#Want the model to like train for 100 times
BUFFER_SIZE = 100

#32 sample of process before updating
BATCH_SIZE = 32

In [ ]:
"""
Read our csv files name the column type (ham or spam), and message.
"""
dftrain = pd.read_csv(train_file_path, sep="\t", header=None, 
                      names=['type', 'message'])

dftest = pd.read_csv(test_file_path, sep="\t", header=None, 
                     names=['type', 'message'])

In [ ]:
"""
Convert our categorical data into numerical data. If the message is a spam, then
1 is given. If ham, then 0 is given.
"""

dftrain['type'] = dftrain['type'].apply(lambda x: 1 if x == 'spam' else 0)
dftest['type'] = dftest['type'].apply(lambda x: 1 if x == 'spam' else 0)
dftrain.head()

In [ ]:
"""
Getting the values of type which is the thing we're predicting
"""
#Store the values of type into our labels
train_labels = dftrain['type'].values
test_labels = dftest['type'].values

#Slicing our dataset with the messages and our train_label
train_ds = tf.data.Dataset.from_tensor_slices((
    dftrain['message'].values, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices(
    (dftest['message'].values, test_labels))

In [ ]:
"""
Before passing our data into the vector and model, we should set the buffer size
"""
#Make the cpu prefetch the batches before passing it on for training
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(
    tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
"""
Set up a vectorizer to turn each word in our dictionary into a vector to know
which words are similar and which words are not.
"""


#Set up a vectorizer to turn words into a numerical vector for training
vectorizer = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

#give our words to the vectorizer
vectorizer.adapt(train_ds.map(lambda text, label: text))

#take a look at what the vectorizer knows
vocab = np.array(vectorizer.get_vocabulary())
vocab[:20]

In [ ]:
"""
Create our model and add the layers in
"""
#create the model
model = tf.keras.Sequential([
    #Set a layer with the vectorizer for training the words
    vectorizer, tf.keras.layers.Embedding(
        len(vectorizer.get_vocabulary()),
        64,
        mask_zero=True,
    ),

    #Deal with the long short term memory
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    
    #Dense our layers into an array of 32
    tf.keras.layers.Dense(32, activation='relu'),
    #What to forget
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])

#compile the model witht the optimizer and the loss function
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

model.summary()

In [ ]:
#train the model
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=6,
)

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    ps = model.predict([pred_text]) #make a prediction
    print(ps)
    p = ps[0][0] #access the probability
    return [p, "ham" if p <0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


***Experimenting with data***

**Spam** 

In [ ]:
dftrain['class_label'].value_counts()

In [ ]:
df_spam = dftrain[dftrain.class_label=='spam']
df_spam

In [ ]:
spam_list= df_spam['message'].tolist()
filtered_spam = ' '.join(spam_list).lower()

In [ ]:
#create and generate a word cloud image
wordcloud = WordCloud(max_font_size = 160, margin=0, background_color = "white", colormap="Reds").generate(filtered_spam)

**Ham**

In [ ]:
df_ham = dftrain[dftrain.class_label=='ham']
df_ham

In [ ]:
ham_list= df_ham['message'].tolist()
filtered_ham = ' '.join(ham_list).lower()

In [ ]:
#create and generate a word cloud image
wordcloud = WordCloud(max_font_size = 160, margin=0, background_color = "white", colormap="Reds").generate(filtered_ham)

End of experimentation